### Passo 1 - Importar Arquivos e Bibliotecas

In [99]:
#Importar Bibliotecas
import pandas as pd
from pathlib import Path
import win32com.client as win32

In [100]:
#Importas Arquivos

emails = pd.read_excel(r'Bases de dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de dados\Vendas.xlsx')
#display(emails)
#display(lojas)
#display(vendas)

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [101]:
# Incluir Lojas em vendas
vendas = vendas.merge(lojas, on='ID Loja')

In [102]:
#Criando tabela para cada Loja
dicionario_loja = {}
for loja in lojas['Loja']:
    dicionario_loja[loja] = vendas.loc[vendas['Loja'] == loja, :]

In [103]:
dia_indicador = vendas['Data'].max()
print(dia_indicador)

2019-12-26 00:00:00


### Passo 3 - Salvar a planilha na pasta de backup

In [104]:
#Identificar se a pasta ja existe
caminho_backup = Path(r'Backup Arquivos Lojas')

pastas_backup = caminho_backup.iterdir()

lista_nomes_backup = [pasta.name for pasta in pastas_backup]

for loja in dicionario_loja:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    
    #salvar dentro da pasta
    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_loja[loja].to_excel(local_arquivo, index=False)

### Passo 4 - Calcular o indicador para 1 loja

In [105]:
# Definindo as metas
meta_faturamento_ano = 1650000
meta_faturamento_dia = 1000
meta_diversidade_ano = 120
meta_diversidade_dia = 4
meta_ticket_ano = 500
meta_ticket_dia = 500

### Passo 5 - Enviar por e-mail para o gerente

In [106]:
for loja in dicionario_loja:
    vendas_loja = dicionario_loja[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]


    # Faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    # Diversidade de produtos
    quant_produtos_ano = len(vendas_loja['Produto'].unique())
    quant_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    # Ticket médio por venda
    valor_por_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_por_venda['Valor Final'].mean()

    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    
    
    #Enviando E-mail
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    #Lógica das cores dos Indicadores
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'

    if quant_produtos_ano >= meta_diversidade_ano:
        cor_div_ano = 'green'
    else:
        cor_div_ano = 'red'
    if quant_produtos_dia >= meta_diversidade_dia:
        cor_div_dia = 'green'
    else:
        cor_div_dia = 'red'

    if ticket_medio_ano >= meta_ticket_ano:
        cor_tik_ano = 'green'
    else:
        cor_tik_ano = 'red'

    if ticket_medio_dia >= meta_ticket_dia:
        cor_tik_dia = 'green'
    else:
        cor_tik_dia = 'red'

    #Corpo do E-mail
    mail.HTMLBody = f'''
    <p>Bom dia, {emails.loc[emails['Loja']==loja, 'Gerente'].values[0]}</p>

    <p>O resultado de ontem <strong>(dia {dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color = "{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{quant_produtos_dia}</td>
        <td style="text-align: center">{meta_diversidade_dia}</td>
        <td style="text-align: center"><font color = "{cor_div_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticket_dia:.2f}</td>
        <td style="text-align: center"><font color = "{cor_tik_dia}">◙</font></td>
      </tr>

    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color = "{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{quant_produtos_ano}</td>
        <td style="text-align: center">{meta_diversidade_ano}</td>
        <td style="text-align: center"><font color = "{cor_div_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticket_ano:.2f}</td>
        <td style="text-align: center"><font color = "{cor_tik_ano}">◙</font></td>
      </tr>

    </table>

    <p>Segue em anexo a planilha com os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou a disposição</p>
    <p>Att., Carlos</p>
    '''

    # Anexos
    attachment  = Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()
    print(f'E-mail da loja {loja} enviado.')

E-mail da loja Iguatemi Esplanada enviado.
E-mail da loja Shopping Midway Mall enviado.
E-mail da loja Norte Shopping enviado.
E-mail da loja Shopping Iguatemi Fortaleza enviado.
E-mail da loja Shopping União de Osasco enviado.
E-mail da loja Shopping Center Interlagos enviado.
E-mail da loja Rio Mar Recife enviado.
E-mail da loja Salvador Shopping enviado.
E-mail da loja Rio Mar Shopping Fortaleza enviado.
E-mail da loja Shopping Center Leste Aricanduva enviado.
E-mail da loja Ribeirão Shopping enviado.
E-mail da loja Shopping Morumbi enviado.
E-mail da loja Parque Dom Pedro Shopping enviado.
E-mail da loja Bourbon Shopping SP enviado.
E-mail da loja Palladium Shopping Curitiba enviado.
E-mail da loja Passei das Águas Shopping enviado.
E-mail da loja Center Shopping Uberlândia enviado.
E-mail da loja Shopping Recife enviado.
E-mail da loja Shopping Vila Velha enviado.
E-mail da loja Shopping SP Market enviado.
E-mail da loja Shopping Eldorado enviado.
E-mail da loja Shopping Ibirapuer

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [107]:
#Faturamento Anual por Loja
faturamento_loja = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_loja_ano = faturamento_loja.sort_values('Valor Final', ascending=False)
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
faturamento_loja_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

#Faturamento do dia por Loja
vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_loja_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_loja_dia = faturamento_loja_dia.sort_values('Valor Final', ascending=False)
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
faturamento_loja_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

### Passo 8 - Enviar e-mail para diretoria

In [108]:
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia!

Melhor loja do dia em faturamento: Loja {faturamento_loja_dia.index[0]}, com faruramento de R${faturamento_loja_dia.iloc[0,0]:.2f}.
Pior loja do dia em faturamento: Loja {faturamento_loja_dia.index[-1]}, com faturamento de R${faturamento_loja_dia.iloc[-1,0]:.2f}.

Melhor loja do ano em faturamento: Loja {faturamento_loja_ano.index[0]}, com faruramento de R${faturamento_loja_ano.iloc[0,0]:.2f}.
Pior loja do ano em faturamento: Loja {faturamento_loja_ano.index[-1]}, com faturamento de R${faturamento_loja_ano.iloc[-1,0]:.2f}.

Segue anexado o rankind do ano e do dia das Lojas.

Qualquer dúvida estou a disposição.

Att.,
Carlos.
'''

# Anexos
attachment  = Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment  = Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print(f'E-mail da Diretoria {loja} enviado.')

E-mail da Diretoria Shopping Barra enviado.
